In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
tabela = pd.read_excel('Contas a Receber.xlsx')

display(tabela)
tabela.info()

In [ ]:
hoje = dt.datetime.now()

print(hoje)

In [ ]:
tabela_devedores = tabela.loc[tabela['Status']=='Em aberto']
display(tabela_devedores)

tabela_devedores = tabela_devedores.loc[tabela_devedores['Data Prevista para pagamento']<hoje]
display(tabela_devedores)

In [ ]:
dados = tabela_devedores[['Valor em aberto','Data Prevista para pagamento', 'E-mail', 'NF']].values.tolist()

display(dados)

In [ ]:
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
our_email = ''

In [ ]:
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_gmail.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

In [ ]:
# a password sempre tem que ir nas configurações da conta do google e criar uma senha para app

import smtplib
import email.message

def enviar_email():  
    for dado in dados:
        destinatario = dado[2] 
        nf=dado[3]
        prazo=dado[1]
        prazo=prazo.strftime("%d/%m/%y")
    
    
        corpo_email = f"""
        <p>Olá {destinatario}, tudo bem? Temos um valor em aberto de referente a nota fiscal {nf}</p>
        <p>O prazo de pagamento era {prazo}</p>
        """

        msg = email.message.Message()
        msg['Subject'] = "Cobrança"
        msg['From'] = ''
        msg['To'] = destinatario
        password = 'mfwhfyuiocbelrzd' 
        msg.add_header('Content-Type', 'text/html')
        msg.set_payload(corpo_email )

        s = smtplib.SMTP('smtp.gmail.com: 587')
        s.starttls()
        # Login Credentials for sending the mail
        s.login(msg['From'], password)
        s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
        print('Email enviado')

In [ ]:
enviar_email()